In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import sys
import time
import json
import numpy as np
import pandas as pd
import geopandas as gpd
import pickle as pkl
import networkx as nx
import matplotlib.pyplot as plt

from pprint import pprint

import src
from src.reload import deep_reload

This notebook shows how supply sensitive routing works using a California's SNG. The core concepts are outlined in the associated paper. A case including vehicle and station parameters is generated and run. Please read README.md befor running.

In [ ]:
'''
Importing California combined SNG
'''

graph = src.graph.graph_from_json('Outputs/sng_combined.json')
graph = graph.to_directed()

In [ ]:
cities = src.graph.graph_from_json('Outputs/places.json')

pop_adj = sum([v['population'] for v in cities._node.values()]) / cities.number_of_nodes()

weighted = {k: v['population'] / pop_adj for k, v in cities._node.items()}

In [ ]:
'''
Generating a case
'''
deep_reload(src)

seed = 1468952
seed = None
rng = np.random.default_rng(seed)

vehicle_param = src.experiments._vehicle_kwargs
station_param = src.experiments._station_kwargs
graphs = [0]

graph_index, vehicle_kw, station_kw = src.experiments.generate_case(
    graphs, vehicle_param, station_param, rng,
)

graph_index, vehicle_kw, station_kw

In [ ]:
'''
Running a case
'''
deep_reload(src)

t0 = time.time()

costs, values, paths = src.experiments.run_case(
    graph, vehicle_kw, station_kw, method = 'dijkstra',
)

print(f'Executed in {time.time() - t0:.4f} seconds')

In [ ]:
'''
Computing weighted impedance
'''

print(src.routing.impedance(
    values, field = 'total_time',
    origins = weighted,
    destinations = weighted,
) / 3600)

print(src.routing.impedance(
    values, field = 'driving_time',
    origins = weighted,
    destinations = weighted,
) / 3600)